In [12]:
# import the Libraries
import numpy as np                   
import pandas as pd                  
import matplotlib.pyplot as plt 

In [13]:
path1 = "/content/drive/MyDrive/Colab Notebooks/Jobathon/train_s3TEQDk.csv"
path2 = '/content/drive/MyDrive/Colab Notebooks/Jobathon/test_mSzZ8RL.csv'
path3 = '/content/drive/MyDrive/Colab Notebooks/Jobathon/sample_submission_eyYijxG.csv'

In [14]:
# read the datsets
train = pd.read_csv(path1)
test = pd.read_csv(path2)
sample = pd.read_csv(path3)

In [15]:
# loosing around 10% data as no correlation was found for Credit_Product with other variables
train['Credit_Product']= train['Credit_Product'].fillna('NA')
train= train.reset_index(drop=True)

In [16]:
# Encoding the categorical data
from sklearn.preprocessing import OneHotEncoder

oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(train[["Occupation"]])
train= train.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_))
train.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,"(Entrepreneur,)","(Other,)","(Salaried,)","(Self_Employed,)"
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0,0.0,1.0,0.0,0.0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0,0.0,0.0,1.0,0.0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0,0.0,0.0,0.0,1.0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0,0.0,0.0,1.0,0.0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0,0.0,0.0,1.0,0.0


In [17]:
# Encoding the categorical data
from sklearn.preprocessing import OneHotEncoder

oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(train[["Channel_Code"]])
train = train.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_))
train.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,"(Entrepreneur,)","(Other,)","(Salaried,)","(Self_Employed,)","(X1,)","(X2,)","(X3,)","(X4,)"
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [18]:
# Encoding the categorical data
from sklearn.preprocessing import OneHotEncoder

oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(train[["Region_Code"]])
train = train.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_))


In [19]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
train['Gender'] = lb_make.fit_transform(train['Gender'])
train['Is_Active'] = lb_make.fit_transform(train['Is_Active'])
train['Credit_Product'] = lb_make.fit_transform(train['Credit_Product'])
train.shape

(245725, 54)

In [20]:
tr = train.drop(['ID','Region_Code','Occupation','Channel_Code'],axis = 1)
X = tr.drop(['Is_Lead'],axis=1)
y = tr[['Is_Lead']]

In [21]:
# split the dataset into test set and train set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size= 0.2, random_state=0)
X.isnull().sum()

Gender                 0
Age                    0
Vintage                0
Credit_Product         0
Avg_Account_Balance    0
Is_Active              0
(Entrepreneur,)        0
(Other,)               0
(Salaried,)            0
(Self_Employed,)       0
(X1,)                  0
(X2,)                  0
(X3,)                  0
(X4,)                  0
(RG250,)               0
(RG251,)               0
(RG252,)               0
(RG253,)               0
(RG254,)               0
(RG255,)               0
(RG256,)               0
(RG257,)               0
(RG258,)               0
(RG259,)               0
(RG260,)               0
(RG261,)               0
(RG262,)               0
(RG263,)               0
(RG264,)               0
(RG265,)               0
(RG266,)               0
(RG267,)               0
(RG268,)               0
(RG269,)               0
(RG270,)               0
(RG271,)               0
(RG272,)               0
(RG273,)               0
(RG274,)               0
(RG275,)               0


In [22]:
from xgboost import XGBClassifier

my_model =  XGBClassifier()
my_model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [23]:
y_pred = my_model.predict(X_test)
y_pred = (y_pred>0.5)
y_pred

array([False, False, False, ..., False, False, False])

In [24]:
# Confusion Metric
from sklearn.metrics import confusion_matrix
confusion_metric = confusion_matrix(y_test, y_pred)
confusion_metric

array([[36123,  1388],
       [ 5431,  6203]])

In [25]:
# accuracy score
from sklearn.metrics import accuracy_score
accuracy_score = accuracy_score(y_test, y_pred)
accuracy_score

0.8612473293315699

In [26]:
from sklearn.metrics import roc_auc_score
roc_auc_score = roc_auc_score(y_test, y_pred)
roc_auc_score

0.7480880675666524

In [27]:
test.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No
3,TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No
4,SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No


In [28]:
test.isnull().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         12522
Avg_Account_Balance        0
Is_Active                  0
dtype: int64

In [29]:
train['Credit_Product'].value_counts()

1    144357
2     72043
0     29325
Name: Credit_Product, dtype: int64

In [31]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(random_state = 0)
forest.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = forest.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[35279  2232]
 [ 5027  6607]]


0.8522942313561909

In [34]:
from sklearn.metrics import roc_auc_score
roc_auc_score = roc_auc_score(y_test, y_pred)
roc_auc_score

0.75420093608253

In [35]:
train.describe()

,Gender,Age,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,"(Entrepreneur,)","(Other,)","(Salaried,)","(Self_Employed,)","(X1,)","(X2,)","(X3,)","(X4,)","(RG250,)","(RG251,)","(RG252,)","(RG253,)","(RG254,)","(RG255,)","(RG256,)","(RG257,)","(RG258,)","(RG259,)","(RG260,)","(RG261,)","(RG262,)","(RG263,)","(RG264,)","(RG265,)","(RG266,)","(RG267,)","(RG268,)","(RG269,)","(RG270,)","(RG271,)","(RG272,)","(RG273,)","(RG274,)","(RG275,)","(RG276,)","(RG277,)","(RG278,)","(RG279,)","(RG280,)","(RG281,)","(RG282,)","(RG283,)","(RG284,)"
count,245725.000000,245725.000000,245725.000000,245725.000000,2.457250e+05,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000,245725.000000
mean,0.546127,43.856307,46.959141,1.173845,1.128403e+06,0.388381,0.237208,0.010854,0.285575,0.293006,0.410565,0.422090,0.275617,0.279630,0.022664,0.010158,0.024214,0.017442,0.007561,0.109228,0.008212,0.011586,0.024829,0.007940,0.010524,0.012656,0.031063,0.007276,0.015005,0.011366,0.006292,0.006422,0.006092,0.146237,0.031999,0.031417,0.006275,0.021373,0.018301,0.021512,0.013206,0.011248,0.052197,0.007415,0.016181,0.051989,0.020726,0.023722,0.119711,0.078624
std,0.497869,14.828672,32.353136,0.618309,8.529364e+05,0.487383,0.425372,0.103614,0.451689,0.455142,0.491937,0.493894,0.446826,0.448818,0.148829,0.100272,0.153713,0.130913,0.086627,0.311925,0.090250,0.107014,0.155603,0.088751,0.102045,0.111787,0.173489,0.084991,0.121571,0.106006,0.079070,0.079879,0.077814,0.353344,0.175998,0.174443,0.078968,0.144626,0.134038,0.145083,0.114156,0.105460,0.222424,0.085790,0.126170,0.222005,0.142467,0.152181,0.324624,0.269152
min,0.000000,23.000000,7.000000,0.000000,2.079000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,30.000000,20.000000,1.000000,6.043100e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,43.000000,32.000000,1.000000,8.946010e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,54.000000,73.000000,2.000000,1.366666e+06,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [38]:
#@title Processing test data

In [39]:
test['Credit_Product']= test['Credit_Product'].fillna('NA')
test= test.reset_index(drop=True)

In [41]:
# Encoding the categorical data
from sklearn.preprocessing import OneHotEncoder

oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(test[["Occupation"]])
test= test.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_))
test.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,"(Entrepreneur,)","(Other,)","(Salaried,)","(Self_Employed,)"
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No,0.0,1.0,0.0,0.0
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NA,925537,No,0.0,1.0,0.0,0.0
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No,0.0,0.0,1.0,0.0
3,TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No,0.0,1.0,0.0,0.0
4,SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No,0.0,1.0,0.0,0.0


In [42]:
# Encoding the categorical data
from sklearn.preprocessing import OneHotEncoder

oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(test[["Channel_Code"]])
test = test.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_))
test.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,"(Entrepreneur,)","(Other,)","(Salaried,)","(Self_Employed,)","(X1,)","(X2,)","(X3,)","(X4,)"
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NA,925537,No,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [43]:
# Encoding the categorical data
from sklearn.preprocessing import OneHotEncoder

oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(test[["Region_Code"]])
test = test.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_))


In [44]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
test['Gender'] = lb_make.fit_transform(test['Gender'])
test['Is_Active'] = lb_make.fit_transform(test['Is_Active'])
test['Credit_Product'] = lb_make.fit_transform(test['Credit_Product'])
test.shape

(105312, 53)

In [46]:
test.shape

(105312, 53)

In [47]:
te = test.drop(['ID','Region_Code','Occupation','Channel_Code'],axis = 1)

In [49]:
test_pred = forest.predict(te)

In [57]:
test_pred
te_pred = pd.DataFrame(test_pred, columns = ['Is_Lead'])

In [51]:
submission = test['ID']

In [52]:
submission

0         VBENBARO
1         CCMEWNKY
2         VK3KGA9M
3         TT8RPZVC
4         SHQZEYTZ
            ...   
105307    DBENJOYI
105308    CWQ72DWS
105309    HDESC8GU
105310    2PW4SFCA
105311    F2NOYPPZ
Name: ID, Length: 105312, dtype: object

In [63]:
final=pd.concat([submission, te_pred], axis = 1)

In [65]:
final.shape

(105312, 2)

In [71]:
final.to_csv ('final.csv', index = False, header=True)
!cp final.csv "/content/drive/MyDrive/Colab Notebooks/Jobathon"